# This notebook creates additional features from the series details and series data csv files that were created from the get_new_data notebook


In [1]:
import pandas as pd
import numpy as np
from fred import Fred
import time
import datetime
import csv
import statsmodels.tsa.stattools
from statsmodels.tsa.stattools import adfuller
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import matplotlib as mpl

In [51]:
# read in the details for all 10868 series
series_dets = pd.read_csv('../data/raw/all_series_details.csv', parse_dates=[4,6,7,9,10], infer_datetime_format=True)

In [54]:
series_dets

,Unnamed: 0,frequency,frequency_short,id,last_updated,notes,observation_end,observation_start,popularity,realtime_end,realtime_start,seasonal_adjustment,seasonal_adjustment_short,title,units,units_short
0,0,Quarterly,Q,XTEXVA01DKQ667S,2019-08-12 20:08:35,OECD descriptor ID: XTEXVA01\r\nOECD unit ID: ...,2019-01-01,1960-01-01,1,2019-08-31,2019-08-31,Seasonally Adjusted,SA,Exports: Value Goods for Denmark,US Dollars Monthly Level,"US $, Monthly Level"
1,0,Quarterly,Q,B139RL1Q225SBEA,2019-08-29 12:59:46,BEA Account Code: B139RL\r\n\r\nFor more infor...,2019-04-01,1947-04-01,1,2019-08-31,2019-08-31,Seasonally Adjusted Annual Rate,SAAR,Real Motor Vehicle Output: Final Sales of Dome...,Percent Change from Preceding Period,% Chg. from Preceding Period
2,0,Quarterly,Q,NNBTFAQ027S,2019-06-06 16:31:42,Source ID: FA114090005.Q\r\n\r\nFor more infor...,2019-01-01,1946-10-01,1,2019-08-31,2019-08-31,Seasonally Adjusted Annual Rate,SAAR,Nonfinancial noncorporate business; total fina...,Millions of Dollars,Mil. of $
3,0,Quarterly,Q,W108RC1Q027SBEA,2019-08-29 13:03:05,BEA Account Code: W108RC\r\n\r\nFor more infor...,2019-04-01,1947-01-01,0,2019-08-31,2019-08-31,Seasonally Adjusted Annual Rate,SAAR,Government consumption expenditures: Own-accou...,Billions of Dollars,Bil. of $
4,0,Quarterly,Q,CAPUTLHITEK2SQ,2019-08-15 13:27:13,"For a given industry, the capacity utilization...",2019-04-01,1967-01-01,11,2019-08-31,2019-08-31,Seasonally Adjusted,SA,"Capacity Utilization: Computers, communication...",Percent of Capacity,% of Capacity
5,0,Monthly,M,FEDMINNFRWG,2019-08-30 19:01:01,The federal minimum wage is the minimum hourly...,2019-08-01,1938-10-01,40,2019-08-31,2019-08-31,Not Seasonally Adjusted,NSA,Federal Minimum Hourly Wage for Nonfarm Worker...,Dollars per Hour,$ per Hour
6,0,Monthly,M,NOCDFNA066MNFRBPHI,2019-08-15 12:46:12,Current New Orders reports the change in new o...,2019-08-01,1968-05-01,2,2019-08-31,2019-08-31,Not Seasonally Adjusted,NSA,Current New Orders; Diffusion Index for FRB - ...,Index,Index
7,0,Monthly,M,WPUSI093011,2019-08-09 12:51:23,NaN,2019-07-01,1967-01-01,1,2019-08-31,2019-08-31,Not Seasonally Adjusted,NSA,Producer Price Index by Commodity for Special ...,Index 1982=100,Index 1982=100
8,0,Monthly,M,CAPUTLG325S,2019-08-15 13:27:06,"For a given industry, the capacity utilization...",2019-07-01,1948-01-01,13,2019-08-31,2019-08-31,Seasonally Adjusted,SA,Capacity Utilization: Nondurable Manufacturing...,Percent of Capacity,% of Capacity
9,0,Quarterly,Q,JPNPISPIG01GPQ,2019-08-12 17:00:02,OECD descriptor ID: PISPIG01 OECD unit ID: GP ...,2019-04-01,1960-04-01,0,2019-08-31,2019-08-31,Not Seasonally Adjusted,NSA,Producer Prices Index: Stage of processing: In...,Growth rate previous period,Growth rate previous period


Do not create percentage difference features for series that are already 'growth' series or 'change' series or series that are chained to a specific year.

In [55]:
ser_perc = series_dets[series_dets.units.str.contains('Percent', case=False)].id.values
ser_grow = series_dets[series_dets.units.str.contains('growth', case=False)].id.values
ser_chg = series_dets[series_dets.units.str.contains('change', case=False)].id.values

In [84]:
ser_idx = series_dets[series_dets.units.str.contains('index', case=False)].id.values
ser_chn = series_dets[series_dets.units.str.contains('chain', case=False)].id.values
ser_chn2 = series_dets[series_dets.title.str.contains('chain', case=False)].id.values
ser_chg2 = series_dets[series_dets.title.str.contains('change', case=False)].id.values

In [140]:
ser_pgc = set(list(ser_grow) + list(ser_chg) +list(ser_chn) +list(ser_chn2)+list(ser_chg2))

In [141]:
len(ser_pgc)

3050

In [142]:
ser_data = pd.read_csv('../data/processed/all_series_data-082919.csv', index_col='date')
sdf = pd.read_csv('../data/processed/sp500_rsales.csv', index_col='date')

# add data for sp500 and retail sales
ser_data = pd.concat([ser_data, sdf], axis=1, sort=False)

# drop series that are all zeros
zero_drop = list(ser_data.sum()[ser_data.sum() == 0].index)
ser_data.drop(zero_drop, axis=1, inplace=True)

# drop this seris beacause it is problematic
ser_data.drop('A191RE1Q156NBEA', axis=1, inplace=True)
ser_data.shape

(619, 10863)

In [143]:
ser_data.isna().sum().sum()

0

In [144]:
# scale data first to 0-1
rec_ser = ser_data.RECESSION.values
ser_data.drop('RECESSION', axis=1, inplace=True)
ser_data = (ser_data - ser_data.min()) / (ser_data.max() - ser_data.min())
ser_data.head()

,LRHUTTMAJPM156S,CANLOLITOAASTSAM,MANMM101ISM657S,DDDFOINS,CES4000000007,PCU333618333618F,IDS1YMAORIAIIS,WPU0284,IRSTCI01SEQ156N,CUURA318SAM,...,NAEXKP07USQ652S,MABMM301INM189N,XTNTVA01GRM664N,LFWA25FEUSM647N,AB67RG3Q086SBEA,PRS88003142,LNS13000319,ULQEUL01FRQ657S,SP500,RRSFS
date,,,,,,,,,,,,,,,,,,,,,
1968-01-01,0.061224,0.624295,0.247070,0.011869,1.000000,0.000000,0.0,0.000000,0.475806,0.000000,...,0.002108,0.000000,0.999988,0.000000,0.0,0.451613,0.111940,0.53032,0.009839,0.000000
1968-02-01,0.061224,0.620568,0.291203,0.008902,1.000000,0.001170,0.0,0.001451,0.475806,0.000000,...,0.002108,0.000005,0.999916,0.001152,0.0,0.451613,0.171642,0.53032,0.008852,0.005216
1968-03-01,0.040816,0.620005,0.241940,0.011869,1.000000,0.001170,0.0,0.001451,0.475806,0.000000,...,0.002108,0.000014,0.999954,0.002304,0.0,0.451613,0.196517,0.53032,0.009140,0.017762
1968-04-01,0.061224,0.625887,0.313992,0.011869,1.000000,0.001170,0.0,0.001451,0.475806,0.000206,...,0.002108,0.000024,0.999980,0.003456,0.0,0.451613,0.233831,0.53032,0.011629,0.014621
1968-05-01,0.020408,0.637924,0.610442,0.011869,0.982143,0.002341,0.0,0.001451,0.475806,0.000206,...,0.002108,0.000023,0.999164,0.004643,0.0,0.451613,0.236318,0.53032,0.012047,0.016720


In [145]:
ser_data.isna().sum().sum()

0

In [146]:
rec_ser = rec_ser[12:]

# add columns for difference from a quarter ago, a year ago, and from the 10month moving average
for col in ser_data.columns:
    if col not in ser_pgc:
        ser_data[col + '_diffQ'] = (ser_data[col] - ser_data[col].shift(3)) / ser_data[col].shift(3)
        ser_data[col + '_diffA'] = (ser_data[col] - ser_data[col].shift(12)) / ser_data[col].shift(12)
        ser_data[col + '_diffMA'] = (ser_data[col] - ser_data[col].rolling(10).mean()) / ser_data[col].rolling(10).mean()
    
# start the data at 1/1/69
# previous data for the new features above
ser_data = ser_data.iloc[12:]

# make sure no NAs in the data
print('nan ',ser_data.isna().sum().sum())
print('inf ',np.isinf(ser_data.values).sum())

nan  264463
inf  66369


In [147]:
# first replace inf with nan
ser_data.replace([np.inf, -np.inf], np.nan, inplace=True)

# fill nan with zeros
ser_data.fillna(0, inplace=True)
print('nan ',ser_data.isna().sum().sum())
print('inf ',np.isinf(ser_data.values).sum())


nan  0
inf  0


In [148]:
ser_data['RECESSION'] = rec_ser
ser_data.index = pd.to_datetime(ser_data.index)
ser_data.head()

,LRHUTTMAJPM156S,CANLOLITOAASTSAM,MANMM101ISM657S,DDDFOINS,CES4000000007,PCU333618333618F,IDS1YMAORIAIIS,WPU0284,IRSTCI01SEQ156N,CUURA318SAM,...,LNS13000319_diffQ,LNS13000319_diffA,LNS13000319_diffMA,SP500_diffQ,SP500_diffA,SP500_diffMA,RRSFS_diffQ,RRSFS_diffA,RRSFS_diffMA,RECESSION
date,,,,,,,,,,,,,,,,,,,,,
1969-01-01,0.020408,0.810435,0.718261,0.011869,0.982143,0.009947,0.0,0.011611,0.435484,0.003921,...,-0.179104,0.222222,-0.307305,-0.010033,0.375262,0.043517,0.183119,0.000000,0.343247,0
1969-02-01,0.040816,0.809665,0.502998,0.008902,0.946429,0.010532,0.0,0.011611,0.435484,0.003921,...,-0.098765,0.057971,-0.055627,-0.228419,0.339659,-0.087118,-0.107380,6.804769,0.320108,0
1969-03-01,0.040816,0.797792,0.599633,0.008902,0.946429,0.010532,0.0,0.011611,0.516129,0.003921,...,0.030303,-0.139241,-0.088472,-0.058284,0.424231,-0.005344,-0.076143,0.715098,-0.054273,0
1969-04-01,0.040816,0.775092,0.580753,0.008902,0.928571,0.010532,0.0,0.011611,0.516129,0.005366,...,0.109091,-0.351064,-0.151599,0.017228,0.183668,0.040560,-0.193317,1.091972,-0.072000,0
1969-05-01,0.061224,0.743442,0.554535,0.008902,0.946429,0.011703,0.0,0.009192,0.516129,0.005366,...,0.410959,0.084211,0.418733,0.154091,0.136027,0.019486,-0.145121,1.081358,0.032090,0


In [149]:
# replace values of greater than 10 (1000 percent change), with nan
ser_data.where(ser_data <= 10, np.nan, inplace=True)

In [150]:
# fill with most recent reasonable value
ser_data.fillna(method='ffill', inplace=True)

In [163]:
# if no recent data, then fill with zero
ser_data.fillna(0, inplace=True)

In [151]:
# limit the rest of the difference data to a 100 percent change
ser_data.clip_upper(1, inplace=True)

In [153]:
ser_data.max()[ser_data.max() > .9].sort_values()

ESPCPIALLMINMEI_diffQ       0.900000
CUURA318SAM_diffQ           0.900000
CUURA318SAS_diffQ           0.900000
PCU339930339930_diffQ       0.900000
CUURA319SACL1_diffQ         0.900000
LFEAAGTTUSQ647S             0.900385
IPB51210N_diffMA            0.901774
B092RC1Q027SBEA_diffMA      0.901830
ZAFB6DBSE01NCCUQ_diffMA     0.901840
NORPROMANMISMEI_diffMA      0.901840
A681RC1Q027SBEA_diffMA      0.901840
ZAFB6DBSE01CXCUQ_diffMA     0.901840
WPU0161_diffMA              0.901944
PCU335911335911_diffMA      0.902174
PDIVCA_diffMA               0.902277
USDIVCA_diffMA              0.902277
LRAC64MAJPM156N_diffMA      0.902289
XTIMVA01GBM664S_diffMA      0.902314
VALIMPGBM052N_diffMA        0.902314
XTIMVA01GBM664N_diffMA      0.902314
FINXTIMVA01CXMLM_diffMA     0.902576
XTIMVA01CAQ188S_diffMA      0.902655
WPU0382_diffMA              0.902937
CAPUTLG316SQ                0.903307
LFWA24TTUSM647N_diffMA      0.903480
LRAC55FEJPQ156N_diffMA      0.903606
LRAC55FEJPM156N_diffMA      0.903758
U

In [167]:
ser_data.shape

(607, 34299)

In [168]:
# dates to drop from rows that are part of reccesion but not within 3 months of the start
dates = list(pd.date_range(start='1970-03-01', end='1970-09-01', freq='MS'))\
      + list(pd.date_range(start='1974-02-01', end='1975-02-01', freq='MS'))\
      + list(pd.date_range(start='1980-04-01', end='1980-06-01', freq='MS'))\
      + list(pd.date_range(start='1981-10-01', end='1982-10-01', freq='MS'))\
      + list(pd.date_range(start='1990-10-01', end='1991-02-01', freq='MS'))\
      + list(pd.date_range(start='2001-06-01', end='2001-10-01', freq='MS'))\
      + list(pd.date_range(start='2008-03-01', end='2009-05-01', freq='MS'))

ser_data2 = ser_data.drop(dates)
rec_df = ser_data2['RECESSION']

In [169]:
# complete dataset before any drops
ser_data2.to_csv('../data/processed/series_data_scaled2-082919.csv')

In [171]:
# non-stationary data
def non_stat(series):
    result = adfuller(series)
    if result[0] > result[4]['10%']:
        return True
    else:
        return False

In [173]:
nstat = ser_data2.apply(non_stat)

In [178]:
stat_drop = list(nstat[nstat].index)
len(stat_drop)

7979

In [157]:
# this is slower, do not use this
stat_drop=[]
for col in ser_data2.columns:
    result = adfuller(ser_data2[col])
    if result[0] > result[4]['10%']:
        stat_drop.append(col)
len(stat_drop)

MissingDataError: exog contains inf or nans

In [179]:
with open('../data/processed/stat_drop2.csv', 'w') as myfile:
    wr = csv.writer(myfile)
    wr.writerow(stat_drop)

In [180]:
# start with complete dataset in ser_data2
print(ser_data2.shape)

(546, 34299)


In [122]:
n_drop = pd.read_csv('../data/processed/nber_drop.csv',header=None)

In [181]:
# drop nber recession dates that are after the fact
ser_data2.drop(n_drop.iloc[:,0], axis=1, errors='ignore', inplace=True)
ser_data2.shape

(546, 34255)

In [129]:
# save dataset with nber drop
ser_data2.to_csv('../data/processed/series_data_scaled2-nd-082919.csv')

In [130]:
delay_drop = pd.read_csv('../data/processed/delay_drop.csv')

In [182]:
# drop data series that are delayed by more than ~34 days
ser_data2.drop(delay_drop.columns, axis=1, inplace=True, errors='ignore')
ser_data2.shape

(546, 19019)

In [183]:
# save dataset with delay drop
ser_data2.to_csv('../data/processed/series_data_scaled2-nd-dd-082919.csv')

In [184]:
# drop data series that are non-stationary per adfuller test
ser_data2.drop(stat_drop, axis=1, inplace=True, errors='ignore')
ser_data2.shape

(546, 14260)

In [185]:
# save dataset with stat drop
ser_data2.to_csv('../data/processed/series_data_scaled2-nd-dd-sd-082919.csv')

In [136]:
ser_data2

,CES4000000007,COMPU1USA,CHEBSBUFT02STSAQ,COREFLEXCPIM157SFRBATL,LFEM25TTUSM647S,FINCPIALLQINMEI,BSCICP03DEM665S,DGDSRGM1M225SBEA,WPU104103,CES0800000007,...,LNS13000319_diffQ,LNS13000319_diffA,LNS13000319_diffMA,SP500_diffQ,SP500_diffA,SP500_diffMA,RRSFS_diffQ,RRSFS_diffA,RRSFS_diffMA,RECESSION
date,,,,,,,,,,,,,,,,,,,,,
1969-01-01,0.982143,0.302537,0.787134,0.343684,0.019267,0.004099,0.924833,0.685185,0.003891,0.976190,...,-0.179104,0.222222,-0.307305,-0.010033,0.375262,0.043517,0.183119,0.000000,0.343247,0.0
1969-02-01,0.946429,0.324008,0.787134,0.743014,0.023032,0.004099,0.938461,0.666667,0.003891,0.928571,...,-0.098765,0.057971,-0.055627,-0.228419,0.339659,-0.087118,-0.107380,1.000000,0.320108,0.0
1969-03-01,0.946429,0.324008,0.909803,0.721846,0.021437,0.004385,0.956887,0.703704,0.003891,0.952381,...,0.030303,-0.139241,-0.088472,-0.058284,0.424231,-0.005344,-0.076143,0.715098,-0.054273,0.0
1969-04-01,0.928571,0.245934,0.909803,0.285554,0.020676,0.004385,0.978616,0.722222,0.003891,0.928571,...,0.109091,-0.351064,-0.151599,0.017228,0.183668,0.040560,-0.193317,1.000000,-0.072000,0.0
1969-05-01,0.946429,0.290826,0.909803,0.125175,0.022883,0.004385,0.991709,0.685185,0.003891,0.952381,...,0.410959,0.084211,0.418733,0.154091,0.136027,0.019486,-0.145121,1.000000,0.032090,0.0
1969-06-01,0.910714,0.309694,0.960313,0.495331,0.024515,0.005147,0.989508,0.740741,0.003891,0.928571,...,0.161765,-0.102273,0.076294,-0.100079,-0.051887,-0.124789,-0.097729,0.190172,-0.169973,0.0
1969-07-01,0.928571,0.284320,0.960313,0.487090,0.026036,0.005147,0.985623,0.703704,0.006809,0.928571,...,-0.098361,-0.427083,-0.223164,-0.295392,-0.172807,-0.254703,-0.199238,-0.100228,-0.273377,0.0
1969-08-01,0.928571,0.277163,0.960313,0.354726,0.029114,0.005147,0.988053,0.666667,0.006809,0.928571,...,-0.281553,0.042254,0.034965,-0.199148,-0.094847,-0.139851,-0.138039,-0.105263,-0.104661,0.0
1969-09-01,0.928571,0.257645,1.000000,0.290021,0.029244,0.005719,0.998383,0.685185,0.007782,0.928571,...,-0.177215,-0.197531,-0.070100,-0.134328,-0.244058,-0.170103,0.277998,0.893171,0.082317,1.0


In [144]:
X = ser_data2.drop(['RECESSION'], axis=1)
X.shape

(546, 19463)

In [73]:
# get list of series that are highly correlated
corr_drop = []
idx = 0
for col in X.columns:
    idx += 1
    ser = X[col]
    for cols in X.columns[idx:]:
        corr = np.corrcoef(ser, X[cols])[0,1]
        if corr > 0.95:
            corr_drop.append(col)
            break
    print(idx, end='\r')

In [74]:
len(corr_drop)

6997

In [75]:
with open('../data/raw/corr_drop.csv', 'w') as myfile:
    wr = csv.writer(myfile)
    wr.writerow(corr_drop)